In [15]:
import math

class Scalar:

    def __init__(self, value, op=None, children=(), backward=lambda g: (0,), label=''):
        self.value = value
        self.label = label
        self.op = op
        self.children = children
        self.backward = backward

    def tanh(self):
        t = math.tanh(self.value)
        return Scalar(t, 'tanh', [self], lambda g: ((1 - t ** 2) * g,))

    def __neg__(self):
        return Scalar(-self.value, '-', [self], lambda g: (-g,))

    def __add__(self, other):
        other = wrap(other)
        return Scalar(self.value + other.value, '+', [self, other], lambda g: (g, g))

    def __radd__(self, other):
        other = wrap(other)
        return other + self

    def __sub__(self, other):
        other = wrap(other)
        return self + (-other)

    def __rsub__(self, other):
        other = wrap(other)
        return other + (-self)

    def __mul__(self, other):
        other = wrap(other)
        return Scalar(self.value * other.value, '*', [self, other], lambda g: (g*other.value, g*self.value))

    def __rmul__(self, other):
        other = wrap(other)
        return other * self

    def __pow__(self, power, modulo=None):
        assert isinstance(power, int | float)
        return Scalar(self.value ** power, '**', [self], lambda g: (power * self.value ** (power-1) * g,))

    def __truediv__(self, other):
        return self * (other ** -1)

    def __rtruediv__(self, other):
        other = wrap(other)
        return other / self

    def __repr__(self):
        return f'Scalar({self.label}={self.value})' if self.label != '' else f'Scalar({self.value})'

def wrap(other):
    return other if isinstance(other, Scalar) else Scalar(other)

def grad(root, params):
    grads = {root: 1}  # Gradient of self with respect to self is 1
    stack = [root]  # Stack of nodes to visit
    while stack:
        node = stack.pop()
        if node.children:
            g = grads[node]
            child_grads = node.backward(g)
            for child, child_grad in zip(node.children, child_grads):
                if child not in grads:
                    grads[child] = 0
                    stack.append(child)
                grads[child] += child_grad
    return [grads[p] for p in params]


In [16]:
a = Scalar(10, label='a')
b = Scalar(2, label='b')
c = Scalar(-3, label='c')

xi = a * b - 2 * c ** 2

gs = grad(xi, [a, b, c])
gs


[2, 10, 12]

In [17]:
import random

class Neuron:

    def __init__(self, dimensions):
        self.dimensions = dimensions
        self.weights = [Scalar(random.uniform(-1, 1), label="weight") for d in range(dimensions)]
        self.bias = Scalar(random.uniform(-1, 1), label="bias")

    def __call__(self, xs):
        assert len(xs) == self.dimensions
        act = sum([w * x for (x, w) in zip(xs, self.weights)], self.bias)
        return act.tanh()

    def parameters(self):
        return self.weights + [self.bias]

class Layer:

    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs
        self.neurons = [Neuron(inputs) for o in range(outputs)]

    def __call__(self, xs):
        assert len(xs) == self.inputs
        return [n(xs) for n in self.neurons]

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

class MLP:

    def __init__(self, inputs, *layers):
        self.inputs = inputs
        self.layers = [Layer(inputs if index == 0 else layers[index - 1], outputs) for index, outputs in enumerate(layers)]

    def __call__(self, xs):
        assert len(xs) == self.inputs
        x = xs
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]


In [18]:
mlp = MLP(2, 4, 4, 3)

x1 = [1, 2]
y1 = [-1, 1, 1]
p1 = mlp(x1)
p1


[Scalar(-0.38735380435995725),
 Scalar(0.7627609273100618),
 Scalar(0.27432736080004705)]

In [23]:
for step in range(20):
    pred = mlp(x1)
    loss = sum([(p - y) ** 2 for p, y in zip(pred, y1)])
    print(f"Loss: {loss.value}")
    params = mlp.parameters()
    grads = grad(loss, params)
    for g, p in zip(grads, params):
        p.value -= 0.1 * g


Loss: 0.01240515781534988
Loss: 0.012112588699634644
Loss: 0.011832979150463224
Loss: 0.011565497684042511
Loss: 0.011309382121942813
Loss: 0.011063932527462541
Loss: 0.010828504988850858
Loss: 0.010602506133136614
Loss: 0.010385388272223881
Loss: 0.01017664509777683
Loss: 0.009975807853794705
Loss: 0.009782441926125412
Loss: 0.009596143796847829
Loss: 0.009416538318761655
Loss: 0.009243276271396728
Loss: 0.009076032165183875
Loss: 0.008914502264875179
Loss: 0.008758402807090336
Loss: 0.008607468390104392
Loss: 0.008461450516769692


In [24]:
p2 = mlp(x1)
p2

[Scalar(-0.9397535845300004),
 Scalar(0.94055753925621),
 Scalar(0.9659841278578938)]